In [1]:
import pandas as pd
from datasets import load_dataset

/Users/tousif/miniforge3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df1 = pd.read_csv("News-Classification-Dataset/barely-true-cleaned.csv")
df2 = pd.read_csv("News-Classification-Dataset/false-cleaned.csv")
df3 = pd.read_csv("News-Classification-Dataset/half-true-cleaned.csv")
df4 = pd.read_csv("News-Classification-Dataset/mostly-true-cleaned.csv")
df5 = pd.read_csv("News-Classification-Dataset/pants-fire-cleaned.csv")
df6 = pd.read_csv("News-Classification-Dataset/true-cleaned.csv")


In [4]:
df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

## Checking the data for imbalances

In [5]:
# Basic overview of the dataset
print(f"Dataset shape: {df.shape}")
df.info()

Dataset shape: (15000, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Statement  15000 non-null  object
 1   Label      15000 non-null  object
dtypes: object(2)
memory usage: 234.5+ KB


In [6]:
# Check for missing values
missing_values = df.isnull().sum()
print("\nMissing values per column:")
print(missing_values[missing_values > 0])  # Only show columns with missing values



Missing values per column:
Series([], dtype: int64)


In [8]:
# Check class distribution 
class_count = df['Label'].value_counts()
print("\nClass count (%):")
print(class_count)


Class count (%):
Label
barelytrue    2500
False         2500
halftrue      2500
mostlytrue    2500
pantsfire     2500
True          2500
Name: count, dtype: int64
